# 🚀 11 - Progetto finale: Gestione di un inventario avanzato

---
## Introduzione al progetto

In questo capitolo, metterai insieme tutti i concetti imparati finora per creare un piccolo programma che gestisce un inventario di prodotti. Questa volta, useremo un approccio più professionale e modulare, sfruttando in particolare la **Programmazione Orientata agli Oggetti** per modellare il problema in modo più efficace.

Utilizzeremo:

- **Ambienti virtuali (`venv`)** per isolare le dipendenze del progetto.
- **Classi e Oggetti (OOP)** per rappresentare i prodotti e l'inventario stesso.
- **Concetti avanzati di OOP** come **ereditarietà**, **incapsulamento** e **polimorfismo**.
- **Librerie esterne** come `colorama` per migliorare l'interfaccia utente.
- **Strutture dati** (dizionari) per organizzare l'inventario all'interno di una classe.
- **Cicli (`while`, `for`)** per l'interazione con l'utente e l'elaborazione dei dati.
- **Gestione degli errori (`try-except`)** per rendere il programma robusto.
- **Input/Output da file** con il modulo `json` per salvare e caricare i dati in modo persistente.

---
## 0. Preparazione del Progetto con Ambienti Virtuali 💻

Prima di scrivere il codice, la buona pratica ci insegna a creare un ambiente isolato per il nostro progetto. Questo ci assicura che le dipendenze non entrino in conflitto con altri progetti sul tuo sistema.

Apri il terminale, naviga nella cartella del progetto ed esegui i seguenti comandi:

1.  **Crea l'ambiente virtuale:**
    `python3 -m venv venv`

2.  **Attiva l'ambiente virtuale:**
    *Su macOS / Linux:* `source venv/bin/activate`
    *Su Windows:* `venv\Scripts\activate.bat`

3.  **Installa la libreria esterna `colorama`:**
    `pip install colorama`

Ora sei pronto per scrivere il codice!

---
## 1. Struttura del Progetto e Gestione delle Dipendenze 📁

Per un'organizzazione chiara e professionale, il progetto è suddiviso in più file. Questo rende il codice più facile da leggere, testare e mantenere.

La struttura della cartella del progetto sarà la seguente:

```
gestore-inventario/
├── venv/                   # Ambiente virtuale
├── inventario.json         # File di salvataggio dei dati
├── requirements.txt        # Dipendenze del progetto
├── main.py                 # Loop principale e interfaccia utente
├── models.py               # Classi per la modellazione dei prodotti
├── inventory_manager.py    # Classe per la gestione dell'inventario
└── tests/                  # Cartella per i test unitari
    └── test_inventory.py   # File di test per la logica del programma
```

### File `requirements.txt`

Questo file elenca le librerie esterne necessarie per il progetto.

```txt
colorama==0.4.6
```

Puoi installare tutte le dipendenze usando pip:

```bash
pip install -r requirements.txt
```

---
## 2. Implementazione del Codice 💻

Di seguito trovi il codice suddiviso nei rispettivi file.

### `models.py`

Questo file contiene le definizioni delle classi **`BaseProdotto`**, **`ProdottoAlimentare`** e **`ProdottoElettronico`**. Queste classi usano l'ereditarietà e le classi astratte per definire una struttura comune e specifica per ogni tipo di prodotto.

In [ ]:
# models.py

from abc import ABC, abstractmethod

# Classe base astratta per tutti i prodotti
class BaseProdotto(ABC):
    def __init__(self, nome: str, prezzo: float, quantita: int):
        self.nome = nome
        self.__prezzo = prezzo  # Attributo privato incapsulato
        self.quantita = quantita

    # Metodo astratto che ogni sottoclasse dovrà implementare
    @abstractmethod
    def __str__(self):
        pass
    
    # Metodo per l'incapsulamento del prezzo
    def get_prezzo(self) -> float:
        return self.__prezzo

    # Metodo per impostare il prezzo, con validazione
    def set_prezzo(self, nuovo_prezzo: float):
        if nuovo_prezzo > 0:
            self.__prezzo = nuovo_prezzo
        else:
            print("Il prezzo deve essere maggiore di zero.")
            
    def to_dict(self):
        return self.__dict__
    
    @classmethod
    def from_dict(cls, data):
        return cls(**data)


# Sottoclasse per prodotti alimentari
class ProdottoAlimentare(BaseProdotto):
    def __init__(self, nome: str, prezzo: float, quantita: int, scadenza: str):
        super().__init__(nome, prezzo, quantita)
        self.scadenza = scadenza
    
    def __str__(self):
        return f"ALIMENTARE: {self.nome.capitalize()} | Prezzo: {self.get_prezzo():.2f}€ | Quantità: {self.quantita} | Scadenza: {self.scadenza}"

# Sottoclasse per prodotti elettronici
class ProdottoElettronico(BaseProdotto):
    def __init__(self, nome: str, prezzo: float, quantita: int, garanzia_anni: int):
        super().__init__(nome, prezzo, quantita)
        self.garanzia_anni = garanzia_anni
        
    def __str__(self):
        return f"ELETTRONICO: {self.nome.capitalize()} | Prezzo: {self.get_prezzo():.2f}€ | Quantità: {self.quantita} | Garanzia: {self.garanzia_anni} anni"

# Classe Prodotto, come riferimento per la versione precedente del progetto
class Prodotto:
    """Classe per rappresentare un singolo prodotto nell'inventario."""
    def __init__(self, nome: str, prezzo: float, quantita: int):
        self.nome = nome
        self.prezzo = prezzo
        self.quantita = quantita

    def __str__(self):
        """Restituisce una rappresentazione in stringa dell'oggetto."""
        return f"Prodotto: {self.nome.capitalize()} | Prezzo: {self.prezzo:.2f}€ | Quantità: {self.quantita}"

    def to_dict(self):
        """Converte l'oggetto in un dizionario per il salvataggio."""
        return self.__dict__

    @classmethod
    def from_dict(cls, data):
        """Crea un oggetto Prodotto da un dizionario."""
        return cls(**data)

### `inventory_manager.py`

Questo file contiene la classe **`GestoreInventario`** che incapsula la logica per la gestione dell'inventario, inclusi l'aggiunta, la vendita, la visualizzazione e la gestione del salvataggio e caricamento dei dati da file.

In [ ]:
# inventory_manager.py

import json
from colorama import Fore, Style
from models import Prodotto, ProdottoAlimentare, ProdottoElettronico

class GestoreInventario:
    """Classe che gestisce tutte le operazioni sull'inventario."""
    NOME_FILE = "inventario.json"
    
    def __init__(self):
        self.inventario = {}
        self.carica_inventario()

    def aggiungi_prodotto(self, prodotto):
        if prodotto.nome in self.inventario:
            print(f"{Fore.YELLOW}Attenzione: Il prodotto {prodotto.nome} esiste già.{Style.RESET_ALL}")
            return False
        else:
            self.inventario[prodotto.nome] = prodotto
            print(f"{Fore.GREEN}Prodotto {prodotto.nome} aggiunto con successo.{Style.RESET_ALL}")
            return True

    def vendi_prodotto(self, nome: str, quantita_venduta: int):
        if nome in self.inventario:
            prodotto = self.inventario[nome]
            if prodotto.quantita >= quantita_venduta:
                prodotto.quantita -= quantita_venduta
                print(f"{Fore.GREEN}{quantita_venduta} {nome} vendute. Rimangono {prodotto.quantita}.{Style.RESET_ALL}")
                return True
            else:
                print(f"{Fore.RED}Errore: Quantità insufficiente. Disponibili: {prodotto.quantita}{Style.RESET_ALL}")
                return False
        else:
            print(f"{Fore.RED}Errore: Il prodotto {nome} non esiste.{Style.RESET_ALL}")
            return False

    def visualizza_inventario(self):
        print("\n--- Inventario attuale ---")
        if not self.inventario:
            print("L'inventario è vuoto.")
        for prodotto in self.inventario.values():
            print(prodotto)
        print("--------------------------")
    
    def salva_inventario(self):
        try:
            with open(self.NOME_FILE, 'w') as f:
                inventario_da_salvare = {}
                for nome, prodotto in self.inventario.items():
                    dati_prodotto = prodotto.to_dict()
                    dati_prodotto['tipo_prodotto'] = prodotto.__class__.__name__
                    inventario_da_salvare[nome] = dati_prodotto
                json.dump(inventario_da_salvare, f, indent=4)
            print(f"{Fore.GREEN}Inventario salvato con successo.{Style.RESET_ALL}")
        except IOError:
            print(f"{Fore.RED}Errore durante il salvataggio del file.{Style.RESET_ALL}")

    def carica_inventario(self):
        try:
            with open(self.NOME_FILE, 'r') as f:
                inventario_caricato = json.load(f)
                self.inventario = {}
                for nome, dati in inventario_caricato.items():
                    tipo_prodotto = dati.pop('tipo_prodotto')
                    if tipo_prodotto == 'ProdottoAlimentare':
                        self.inventario[nome] = ProdottoAlimentare(**dati)
                    elif tipo_prodotto == 'ProdottoElettronico':
                        self.inventario[nome] = ProdottoElettronico(**dati)
                    elif tipo_prodotto == 'Prodotto':
                        self.inventario[nome] = Prodotto(**dati)
            print(f"{Fore.GREEN}Inventario caricato con successo.{Style.RESET_ALL}")
        except (IOError, json.JSONDecodeError, TypeError) as e:
            print(f"{Fore.YELLOW}Nessun inventario esistente o file corrotto, ne creo uno nuovo. Dettaglio errore: {e}{Style.RESET_ALL}")
            self.inventario = {}

### `main.py`

Questo è il file principale del programma. Importa la classe **`GestoreInventario`** e le classi dei prodotti e contiene il loop principale che gestisce il menu di interazione con l'utente.

In [ ]:
# main.py

from colorama import Fore, Style
from inventory_manager import GestoreInventario
from models import Prodotto, ProdottoAlimentare, ProdottoElettronico

def menu():
    """Mostra il menu e gestisce l'input dell'utente."""
    gestore = GestoreInventario()
    while True:
        gestore.visualizza_inventario()
        print("\nOpzioni: (a)ggiungi, (v)endi, (s)alva, (r)icarica, (q)uit")
        scelta = input("Scegli un'opzione: ").lower()

        if scelta == 'a':
            tipo = input("Che tipo di prodotto vuoi aggiungere? (a)limentare, (e)lettronico o (s)emplice: ").lower()
            nome = input("Nome prodotto: ")
            try:
                prezzo = float(input("Prezzo: "))
                quantita = int(input("Quantità: "))
                if prezzo <= 0 or quantita <= 0:
                    print(f"{Fore.RED}Errore: Prezzo e quantità devono essere maggiori di zero.{Style.RESET_ALL}")
                    continue
                
                if tipo == 'a':
                    scadenza = input("Data di scadenza (gg-mm-aaaa): ")
                    nuovo_prodotto = ProdottoAlimentare(nome, prezzo, quantita, scadenza)
                elif tipo == 'e':
                    garanzia = int(input("Garanzia (anni): "))
                    nuovo_prodotto = ProdottoElettronico(nome, prezzo, quantita, garanzia)
                elif tipo == 's':
                    nuovo_prodotto = Prodotto(nome, prezzo, quantita)
                else:
                    print(f"{Fore.RED}Tipo di prodotto non valido.{Style.RESET_ALL}")
                    continue
                
                gestore.aggiungi_prodotto(nuovo_prodotto)
            except ValueError:
                print(f"{Fore.RED}Errore: Prezzo, quantità e garanzia devono essere numeri validi.{Style.RESET_ALL}")
        
        elif scelta == 'v':
            nome = input("Nome prodotto da vendere: ")
            try:
                quantita_venduta = int(input("Quantità da vendere: "))
                if quantita_venduta <= 0:
                    print(f"{Fore.RED}Errore: La quantità da vendere deve essere maggiore di zero.{Style.RESET_ALL}")
                else:
                    gestore.vendi_prodotto(nome, quantita_venduta)
            except ValueError:
                print(f"{Fore.RED}Errore: La quantità deve essere un numero intero.{Style.RESET_ALL}")

        elif scelta == 's':
            gestore.salva_inventario()
        
        elif scelta == 'r':
            gestore.carica_inventario()

        elif scelta == 'q':
            print("Grazie, programma terminato.")
            break

        else:
            print(f"{Fore.RED}Opzione non valida, riprova.{Style.RESET_ALL}")
            
if __name__ == "__main__":
    menu()

---
## 3. Esecuzione del Progetto e Simulazione 🚀

Una volta che hai organizzato i file e installato le dipendenze, puoi eseguire il programma e testarne le funzionalità.

#### Come avviare il programma

1.  Assicurati che l'ambiente virtuale sia attivo. Se non lo è, usa il comando appropriato per il tuo sistema operativo:
    *Su macOS / Linux:* `source venv/bin/activate`
    *Su Windows:* `venv\Scripts\activate.bat`
2.  Esegui il file principale dalla riga di comando:
    `python main.py`

#### Simulazione di un flusso di lavoro tipico

Dopo aver avviato il programma, puoi interagire con il menu per simulare le operazioni di un magazzino. Di seguito è riportato un esempio di sequenza di comandi che potresti utilizzare:

1.  **Aggiungi un prodotto alimentare:**
    * Scegli l'opzione `a` (aggiungi).
    * Seleziona il tipo `a` (alimentare).
    * Inserisci nome, prezzo, quantità e data di scadenza (es. "Latte", 1.50, 10, "30-12-2025").
2.  **Aggiungi un prodotto elettronico:**
    * Scegli l'opzione `a` (aggiungi).
    * Seleziona il tipo `e` (elettronico).
    * Inserisci nome, prezzo, quantità e anni di garanzia (es. "Smartphone", 799.99, 5, 2).
3.  **Visualizza l'inventario:**
    * Il programma visualizzerà l'inventario automaticamente a ogni ciclo. Controlla che i prodotti appena aggiunti siano presenti.
4.  **Vendi un prodotto:**
    * Scegli l'opzione `v` (vendi).
    * Inserisci il nome del prodotto (es. "Latte").
    * Inserisci la quantità da vendere (es. 2). Il programma ti mostrerà la quantità rimanente.
5.  **Salva l'inventario:**
    * Scegli l'opzione `s` (salva). Il programma salverà lo stato corrente dell'inventario nel file `inventario.json`.
6.  **Esci e riavvia per caricare i dati salvati:**
    * Scegli l'opzione `q` (quit) per terminare il programma.
    * Esegui di nuovo `python main.py`. Il programma si avvierà caricando automaticamente i dati da `inventario.json`. L'inventario visualizzato all'inizio conterrà i prodotti che hai salvato.
7.  **Tenta di vendere un prodotto con quantità insufficiente:**
    * Scegli l'opzione `v` (vendi).
    * Inserisci il nome di un prodotto (es. "Smartphone").
    * Inserisci una quantità superiore a quella disponibile (es. 10). Il programma mostrerà un messaggio di errore.

---
## 4. Test Unitari con `unittest` 🧪

Ora che il progetto è stato implementato, è fondamentale testarne le funzionalità in modo automatico. Aggiungeremo una cartella `tests` alla struttura del progetto, dove scriveremo i nostri test unitari per la classe `GestoreInventario`.

### Struttura dei test

La struttura aggiornata del progetto includerà la cartella `tests/` con un file di test al suo interno:

```
gestore-inventario/
├── ...
├── inventory_manager.py
├── models.py
└── tests/                  # Nuova cartella
    └── test_inventory.py   # Nuovo file di test
```

### Esercizio: Scrivere un test per la classe `GestoreInventario`

Crea il file `tests/test_inventory.py` e scrivi i test per verificare il corretto funzionamento dei metodi chiave della classe `GestoreInventario`.

**Istruzioni:**

1.  Crea una classe `TestGestoreInventario` che erediti da `unittest.TestCase`.
2.  Usa il metodo `setUp` per inizializzare un'istanza di `GestoreInventario` prima di ogni test. Inoltre, aggiungi un blocco per rimuovere il file `inventario.json` per assicurarti che ogni test parta da uno stato pulito.
3.  Scrivi i seguenti test:
    -   `test_aggiungi_prodotto`: verifica che l'aggiunta di un nuovo prodotto funzioni e che la dimensione dell'inventario aumenti.
    -   `test_aggiungi_prodotto_esistente`: verifica che non sia possibile aggiungere un prodotto con lo stesso nome e che la quantità non venga modificata.
    -   `test_vendi_prodotto_successo`: verifica che la quantità di un prodotto si riduca correttamente dopo una vendita.
    -   `test_vendi_prodotto_quantita_insufficiente`: verifica che il programma gestisca correttamente i tentativi di vendere più prodotti di quanti ce ne siano in magazzino.
    -   `test_vendi_prodotto_non_esistente`: verifica che il programma gestisca un tentativo di vendita di un prodotto non presente.
    -   `test_salva_carica_inventario`: verifica che i dati vengano salvati e ricaricati correttamente, mantenendo lo stato dell'inventario.
4.  Aggiungi `if __name__ == '__main__':` per poter eseguire i test direttamente.

### Soluzione

Copia il seguente codice nel file `tests/test_inventory.py` per avere tutti i test completi e funzionanti.

In [ ]:
# test_inventory.py
import unittest
import os
from inventory_manager import GestoreInventario
from models import Prodotto, ProdottoAlimentare, ProdottoElettronico

class TestGestoreInventario(unittest.TestCase):

    def setUp(self):
        # Viene eseguito prima di ogni test. 
        # Assicura un ambiente di test pulito eliminando il file di salvataggio.
        self.gestore = GestoreInventario()
        if os.path.exists(self.gestore.NOME_FILE):
            os.remove(self.gestore.NOME_FILE)
        # Inizializza un nuovo gestore che inizierà con un inventario vuoto
        self.gestore = GestoreInventario()
        self.prodotto_test = Prodotto("Test Prodotto", 10.0, 5)

    def tearDown(self):
        # Viene eseguito dopo ogni test. 
        # Pulisce l'ambiente eliminando il file di salvataggio creato.
        if os.path.exists(self.gestore.NOME_FILE):
            os.remove(self.gestore.NOME_FILE)

    def test_aggiungi_prodotto_successo(self):
        self.assertTrue(self.gestore.aggiungi_prodotto(self.prodotto_test))
        self.assertEqual(len(self.gestore.inventario), 1)
        self.assertIn("Test Prodotto", self.gestore.inventario)

    def test_aggiungi_prodotto_esistente(self):
        self.gestore.aggiungi_prodotto(self.prodotto_test)
        prodotto_duplicato = Prodotto("Test Prodotto", 12.0, 3)
        self.assertFalse(self.gestore.aggiungi_prodotto(prodotto_duplicato))
        self.assertEqual(self.gestore.inventario['Test Prodotto'].quantita, 5)

    def test_vendi_prodotto_successo(self):
        self.gestore.aggiungi_prodotto(self.prodotto_test)
        self.assertTrue(self.gestore.vendi_prodotto("Test Prodotto", 3))
        self.assertEqual(self.gestore.inventario['Test Prodotto'].quantita, 2)

    def test_vendi_prodotto_quantita_insufficiente(self):
        self.gestore.aggiungi_prodotto(self.prodotto_test)
        self.assertFalse(self.gestore.vendi_prodotto("Test Prodotto", 10))
        self.assertEqual(self.gestore.inventario['Test Prodotto'].quantita, 5)

    def test_vendi_prodotto_non_esistente(self):
        self.assertFalse(self.gestore.vendi_prodotto("Prodotto Inesistente", 1))

    def test_salva_e_carica_inventario(self):
        prodotto_ali = ProdottoAlimentare("Latte", 1.50, 10, "30-12-2025")
        prodotto_ele = ProdottoElettronico("Laptop", 1200.0, 2, 2)
        self.gestore.aggiungi_prodotto(prodotto_ali)
        self.gestore.aggiungi_prodotto(prodotto_ele)
        self.gestore.salva_inventario()

        # Crea una nuova istanza di GestoreInventario per simulare un nuovo avvio del programma
        nuovo_gestore = GestoreInventario()

        # Verifica che l'inventario sia stato caricato correttamente
        self.assertIn("Latte", nuovo_gestore.inventario)
        self.assertIn("Laptop", nuovo_gestore.inventario)
        self.assertEqual(nuovo_gestore.inventario['Latte'].quantita, 10)
        self.assertEqual(nuovo_gestore.inventario['Laptop'].get_prezzo(), 1200.0)
        self.assertIsInstance(nuovo_gestore.inventario['Latte'], ProdottoAlimentare)
        self.assertIsInstance(nuovo_gestore.inventario['Laptop'], ProdottoElettronico)

if __name__ == '__main__':
    unittest.main()
